In [94]:
import time

import pymongo
from pymongo import MongoClient

from multiprocessing import Process

import subprocess

import requests
from bs4 import BeautifulSoup, NavigableString
from datetime import datetime

import pymongo
from pymongo import MongoClient


import nltk,  operator, re
import pandas as pd
import numpy as np
from konlpy.tag import Twitter, Kkma
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from keras.layers import *
from keras.models import *
from keras.utils import *
from sklearn.preprocessing import *

%matplotlib inline
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go

import logging

In [95]:
def get_html(url):
    _html = ""
    resp = requests.get(url)
    resp.encoding='utf-8'
    if resp.status_code == 200 :
        _html = resp.text
    return _html

In [105]:
#connect mongoDB

def init_mongo():
    global Conn
    Conn = MongoClient('52.79.249.174',27017)
    global db
    db = Conn.crawling
    global background
    background = db.background
    global foreground
    foreground = db.foreground
    global user_tfdif
    user_tfdif = db.user_tfidf
    #Conn.close()

In [110]:
#정치 경제 스포츠 연예 사회 국제
#politics economy sports enter national inter
#   0         1     2     3      4       5
#조선일보 경제는 biz.chosun.com...
CHOSUN = [("http://news.chosun.com/politics",'politics',0),("http://news.chosun.com/sports",'sports',2), ("http://news.chosun.com/ent",'enter',3), ("http://news.chosun.com/national",'national',4),( "http://news.chosun.com/international",'inter',5)]

#동아일보는 패턴 통일
DONGA = [("http://news.donga.com/Politics",'politics',0),("http://news.donga.com/Economy",'economy',1),("http://news.donga.com/Sports",'sports',2),("http://news.donga.com/Enter",'enter',3),("http://news.donga.com/Society",'national',4),("http://news.donga.com/Inter",'inter',5)]

#중앙일보 연예가 가요, 방송, 영화로 나눠져있음
JOONGANG = [("http://news.joins.com/politics",'politics',0),("http://news.joins.com/money",'economy',1),("http://news.joins.com/sports",'sports',2),("http://news.joins.com/society",'national',4),("http://news.joins.com/world",'inter',5)]
JOONGANG_CUL = [("http://news.joins.com/culture/song/list",'enter',3),("http://news.joins.com/culture/broadcast/list",'enter',3),("http://news.joins.com/culture/movie/list",'enter',3)]

#기사 내용을 담을 변수
try:
    _id = foreground.find().sort([('_id',pymongo.DESCENDING)])[0]['_id']
except:
    _id = 0
num_press = 3
num_class = 6
docs = [[] for i in range(num_class)]

lock = asyncio.Lock()

    중앙일보 날짜 가져오기 힘들어서 datetime 패키지를 이용하여 임의로 날짜 입력

In [99]:
def chosun_crawling():
    data_foreground = []
    i = 0
    global _id
    for clas in CHOSUN:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        for lt in soup.find_all("dt") :
            try :
                tmp = lt.find('a')
                link = tmp.get('href')
                day = link.split('/')
                day = day[6]+day[7]+day[8]
                text = tmp.get_text()
                data_foreground.append({'_id': str(_id).zfill(10), 'title' : text, 'press' : 'chosun',  'link' : link, 'day' : day, 'class' : clas[1]})
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class' : 'par'}).get_text(),link), _id)
                _id+=1
            except : 
                print('error!!!',link)
        #docs[i].append(tmp_list)
        i = i + 1
        if i == 1 :
            i = i + 1
    foreground.insert_many(data_foreground)
    print('{0} done!',format(current_process().name))
    return

def chosun_biz():
    html = get_html('http://biz.chosun.com')
    print('http://biz.chosun.com')
    soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
    data_foreground = []
    head = soup.find("div", attrs={'class' : 'mt_art_tit'})
    tmp = head.find('a')
    link = tmp.get('href')
    day = link.split('/')
    day = day[6]+day[7]+day[8]
    text = tmp.get_text()
    global _id
    j=0
    data_foreground.append({'_id': str(_id).zfill(10), 'title' : text, 'press' : 'chosun',  'link' : link, 'day' : day, 'class' : 'economy'})
    
    html = get_html(link)
    soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
    docs[1].append((soup2.find('div',id = 'article_2011', attrs={'class':'article'}).get_text(),link),_id)
    _id+=1
    art = soup.find("div", attrs={'class' : 'mc_art_lst'})
    for lt in art.find_all("li", limit = 30) :
        try :
            tmp = lt.find('a')
            #print(tmp)
            link = tmp.get('href')
            day = link.split('/')
            day = day[6]+day[7]+day[8]
            text = tmp.get_text()
            data_foreground.append({'_id':str(_id).zfill(10), 'title' : text, 'press' : 'chosun',  'link' : link, 'day' : day,'class':'economy'})
            
            html = get_html(link)
            soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
            docs[1].append((soup2.find('div',id = 'article_2011', attrs={'class':'article'}).get_text(),link),_id)
            _id+=1
        except : 
            print('error!!!',link)
    foreground.insert_many(data_foreground)
    print('{0} done!',format(current_process().name))
    return 


In [100]:
def donga_crawling():
    data_foreground = []
    #docs = []
    i = 0
    global _id
    for clas in donga:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        try:
            tmp = soup.find('div',attrs={'class':'articleTop'})
            head = tmp.find("div", attrs={'class' : 'articleMain'})
            #해드라인_main
            head_main = head.find('a')
            link = head_main.get('href')
            day = link.split('/')[6]
            text = head.find(attrs={'class' : 'title'}).get_text()
            data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
            
            #print(data_foreground)
            html = get_html(link)
            soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
            docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link),_id)
            _id+=1
            for head_sub in tmp.find_all('li') :
                art = head_sub.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.find(attrs={'class':'title'}).get_text()
                data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link),_id)
                _id+=1

        except :
            for head in tmp.find_all('div',attrs={'class':'artivleMain'}):
                art = head.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.find(attrs={'class':'title'}).get_text()
                data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link),_id)
                _id+=1
                
            for head in tmp.find_all('div',attrs={'class':'artivleMain02'}):
                art = head.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.find(attrs={'class':'title'}).get_text()
                data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link),_id)
                _id+=1

        try:
            #issue
            content_issue = soup.find('div',attrs={'class':'issueList'})
            issue = content_issue.find('a', attrs={'class' : 'tit'})
            link = issue.get('href')
            day = link.split('/')[6]
            text = issue.get_text();
            data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
            
            html = get_html(link)
            soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
            docs[i].append((soup2.find('div',attrs={'class':'atrticle_txt'}).get_text(),link),_id)
            _id+=1
            for issue_sub in content_issue.find_all('li') :
                art = issue_sub.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.get_text()
                data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link),_id)
                _id+=1
        except :
            print('error!!!',link)
            
        try:
            #최신기사
            contents = soup.find('div',attrs={'class':'articleList_con'})
            for art in contents.find_all('div',attrs={'class':'rightList'}) :
                tmp = art.find('a')
                link = tmp.get('href')
                day = link.split('/')[6]
                text = tmp.find(attrs={'class':'tit'}).get_text()
                data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link),_id)
                _id+=1
        except :
            print('error!!!',link)
        i = i + 1

    foreground.insert_many(data_foreground)
    print('{0} done!',format(current_process().name))
    return 

In [101]:
def joongang_crawling():
    day = datetime.today().strftime("%Y%m%d")
    data_foreground = []
    i = 0
    global _id
    for clas in JOONGANG:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        origin = soup.find('div',id='content')
        #해드라인
        try:
            art = origin.find('dt')
            tmp = art.find('a')
            link = tmp.get('href')
            text = tmp.get_text()
            data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
            
            html2 = get_html(link)
            soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
            docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link),_id)
            _id+=1
        except :
            #sport 해드라인
            try:
                for art in origin.find_all('div',attrs={'class':'slide'}):
                    tmp = art.find(attrs={'class':'headline mg'})
                    tmp2 = tmp.find('a')
                    link = tmp2.get('href')
                    text = tmp2.get_text();
                    data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
                    
                    html2 = get_html(link)
                    soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
                    docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link),_id)
                    _id+=1
            except:
                print('error!!!',link)
        
        #실시간 주요뉴스
        try:
            art = origin.find('div',attrs={'class':'default_realtime'})
            art = art.find('ul',id='ulItems')
            #print(art)
            for content in art.find_all('li'):
                #print(content)
                tmp = content.find(attrs={'class':'headline mg'})
                tmp2 = tmp.find('a')
                link = 'http://news.joins.com/' + tmp2.get('href')
                text = tmp2.get_text()
                data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
                
                html2 = get_html(link)
                soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link),_id)
                _id+=1
        except:
            try:
                art = origin.find('div',attrs={'class':'combination_today'})
                art = art.find('div',attrs={'class':'bd'})
                for content in art.find_all('li'):
                    tmp = content.find(attrs={'class':'headline mg'})
                    tmp2 = tmp.find('a')
                    link = 'http://news.joins.com/' + tmp2.get('href')
                    text = tmp2.get_text()
                    data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
                    
                    html2 = get_html(link)
                    soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
                    docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link),_id)
                    _id+=1
            except :
                print('error!!!',link)
        i = i + 1
        if i == 3 :
            i = i + 1
    foreground.insert_many(data_foreground)
    print('{0} done!',format(current_process().name))
    return 

def joongang_cul():
    day = datetime.today().strftime("%Y%m%d")
    dosc = []
    global _id
    #tmp_list = []
    data_foreground = []
    for clas in JOONGANG_CUL:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        
        origin = soup.find('div',id='content')
        for content in origin.find_all('li'):
            art = content.find(attrs={'class':'headline mg'})
            tmp = art.find('a')
            link = 'http://news.joins.com' + tmp.get('href')
            text = tmp.get_text()
            data_foreground.append({'_id':str(_id).zfill(10),'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
            html2 = get_html(link)
            soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
            docs[3].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link),_id)
            _id+=1
    foreground.insert_many(data_foreground)
    print('{0} done!',format(current_process().name))
    return



    scikit-Learn 패키지를 이용한 TF-IDF 계산


In [102]:
stopwords = ['.',',','\n','\xa0',re.compile('^A-Za-z*$')]


def cal_weight():
    print('calculating weight...')
    tfidf_list = []
    tfidf_list_raw = []
    t = Twitter()
    k = Kkma()
    for i in range(num_class):
    #for i in range(1):
        nouns = []
        for article in docs[i]:
            if article[0] is not '':
                nouns.append(' '.join([noun for noun in t.nouns(str(article[0]))]))        
        vec = TfidfVectorizer(stop_words = stopwords)
        fitted = vec.fit(nouns)
        tfidf_res = fitted.transform(nouns)
        vocab = fitted.get_feature_names()
        j = 0
        for article in tfidf_res.toarray():
            idf = sorted(zip(vocab,article), key=lambda kv:kv[1])[-3:]
            tmp = idf[0][0]+' '+idf[1][0] + ' ' + idf[2][0]
            tfidf_list.append({'_id':docs[i][j][2],'tfidf':tmp,'link':docs[i][j][1]})
            j = j + 1
    background.insert_many(tfidf_list)
    
    print('cal_weight done!')
    return tfidf_list_raw

In [103]:
class MLP:
    #load model
    def loadModel():
        a = False
        date = input("input date you want to load model format : (yyyymmdd) : ")
        self.model = load_model('relu_model'+date)
        self.sig_model = load_model('sigmoid_model'+date)
        self.model_bi = load_model('bi_relu_model'+date)
        self.model_sig_bi = load_model('bi_sigmoid_model'+date)
        if(self.model):
            print('relu model loaded')
            a = True
        else:
            print('No model relu')
        if(self.sig_model):
            print('sigmoid model loaded')
            a = True
        else:
            print('No model sigmoid')
        if(self.model_bi):
            print('bi relu model loaded')
            a = True
        else:
            print('No model bi relu')
            
        if(self.model_sig_bi):
            print('bi sigmoid model loaded')
            a = True
        else:
            print('No model bi sigmoid')
            
        return a
    
    # bidirectional 모델 만들기!
    #activate = 'tanh'
    #kernel_init = 'glorot_uniform'
    #time_step,  output_shape, model_name_to_save
    #number of feature = 8
    split_ratio = 0.8
    max_pad = 15
    def build_model(self, max_pad, categori_shape):
        activate = 'relu'
        kernel_init = 'Orthogonal'
        model = Sequential()
        model.add(LSTM(256, return_sequences=True, input_shape=(max_pad,8), activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(Dense(categori_shape, activation='sigmoid', kernel_initializer = 'Orthogonal'))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        model.summary()

        return model
    
    def build_model_sigmoid(self, max_pad, categori_shape):
        activate_1 = 'sigmoid'
        kernel_init = 'glorot_uniform'
        model = Sequential()
        model.add(LSTM(256, return_sequences=True, input_shape=(max_pad,8), activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init))
        model.add(Dropout(0.2))
        model.add(Dense(categori_shape, activation=activate_1, kernel_initializer = kernel_init))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        model.summary()

        return model
    
    def build_model_bi(self, max_pad, categori_shape):
        activate = 'relu'
        kernel_init = 'Orthogonal'
        model = Sequential()
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init), input_shape=(max_pad,8)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Dense(categori_shape, activation='sigmoid', kernel_initializer = 'Orthogonal'))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        model.summary()

        return model
    
    def build_model_sigmoid_bi(self, max_pad, categori_shape):
        activate_1 = 'sigmoid'
        kernel_init = 'glorot_uniform'
        model = Sequential()
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init), input_shape=(max_pad,8)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(256, return_sequences=True, activation=activate_1, kernel_initializer = kernel_init)))
        model.add(Dropout(0.2))
        model.add(Dense(categori_shape, activation=activate_1, kernel_initializer = kernel_init))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        model.summary()

        return model

    def learning(self, model, train_X, train_Y, test_X, test_Y, model_name):
        history = model.fit(train_X,train_Y, epochs=150, batch_size=1, validation_split=0.1)
        score = model.evaluate(test_X, test_Y)
        model.save(datetime.today().strftime("%Y%m%d")+model_name)
        return (history, score)
    
    def plt_init(self):
        plt.style.use('bmh')

    #plot 그리기
    def draw_plot(self, history, score, name):
        plt.figure(figsize=(6,6))
        plt.title(name)
        plt.plot(history.history['loss'], label = name)
        plt.legend()
        plt.plot(history.history['val_loss'], label = 'val_'+name)
        plt.legend()
        plt.show()
        fig = plt.gcf()
        fig.savefig(datetime.today().strftime("%Y%m%d")+'_'+name, bbox_inches='tight')
        print("loss over test data : %.2f" % (score[0]))
        print("accuracy over test data : %.2f" % (score[1]))

    def data_manipulate(self):
        print('Creating training set...')

        
        #db에서 데이터 불러오기
        user_tfidf_info = user_tfidf.find()
        user_pri_info = user_pri.find()

        #tokenizer 
        token = Tokenizer()

        #db에서 불러온 데이터 dataframe형태로 저장
        user_tfidf = pd.DataFrame(user_tfidf_info,columns = ['ID','tfidf'])
        user_pri = pd.DataFrame(user_pri_info,columns = ['ID','age','gender','inter1','inter2','inter3'])

        #tfidf 분할을 위해 따로 drop
        user_tfidf_value = user_tfidf.drop(columns = ['ID']).values
        user_tfidf = user_tfidf.drop(['tfidf'],axis=1)

        i = 0
        tmp = []
        #tfidf 분할
        for arg in user_tfidf_value:
            tmp.append(' '.join(arg).split(' '))

        #tokenizer 학습
        token.fit_on_texts(tmp)

        tmp = token.texts_to_sequences(tmp)
        
        self.idx_word = {}
        
        for w in token.word_index : 
            self.idx_word[token.word_index[w]] = w

        #분할 한 tfidf dataframe으로 만든 후 원본과 합친다
        tmp = pd.DataFrame(tmp,columns = ['tfidf','tfidf2','tfidf3'])

        user_tfidf = pd.concat([user_tfidf,tmp],axis=1)

        #tfidf와 사용자 정보를 merge
        merged = pd.merge(user_tfidf,user_pri)

        #ID별로 Y 구하기 위해 추출
        ID = user_tfidf.ID.unique()

        train_X = []
        train_Y = []
        
        #train_X max_pad 만큼 ID별로 데이터 추가하거나 삭제
        for id in ID:
            tmp = merged.loc[merged['ID'] == id,'tfidf':].values.T
            tmp = pad_sequences(tmp,maxlen = self.max_pad).T
            train_X.extend(tmp)

        train_X = np.array(train_X)[:,np.newaxis].reshape(-1,self.max_pad,8)

        #ID별로 train_Y

        for id in ID:
            user_private = merged.loc[merged['ID'] == id, 'tfidf' : 'tfidf3' ].values
            user_private = np.vstack((user_private[1:],user_private[0])).T
            user_private = pad_sequences(user_private, maxlen = self.max_pad).T
            train_Y.extend(user_private)

        tmp = []
        for target in to_categorical(np.array(train_Y).reshape(-1,3)):
            tmp.append(sum(target))

        train_Y = np.array(tmp)[:,np.newaxis].reshape(-1,self.max_pad,len(tmp[0]))
        #print(train_Y.shape)

        self.train_X = train_X
        self.train_Y = train_Y
        print('training set created, ready to build model')
        return (train_X, train_Y)
    
    def build_learn_all(self):
        #모델 구축
        split = int(len(self.train_X)*self.split_ratio)
        self.model = self.build_model(self.max_pad, self.train_Y.shape[2])
        self.model_sig = self.build_model_sigmoid(self.max_pad, self.train_Y.shape[2])
        self.model_bi = self.build_model_bi(self.max_pad, self.train_Y.shape[2])
        self.model_sig_bi = self.build_model_sigmoid_bi(self.max_pad, self.train_Y.shape[2])
        
        #모델 run
        (history, score) = self.learning(self.model, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_relu')
        (history2, score2) = self.learning(self.model_sig, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_sigmoid')
        (history3, score3) = self.learning(self.model_bi, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_relu_bi')
        (history4, score4) = self.learning(self.model_sig_bi, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_sigmoid_bi')
        #plot iniit
        plt = self.plt_init()

        #plot 그리기
        self.draw_plot(history, score, '_relu')
        self.draw_plot(history2, score2,'_sigmoid')
        self.draw_plot(history3, score3,'_relu_bi')
        self.draw_plot(history4, score4,'_sigmoid_bi')

        return (self.model, self.model_sig, self.model_bi, self.model_sig_bi);
    
    def build_learn_model(self):
        print('building relu model...')
        split = int(len(self.train_X)*self.split_ratio)
        self.model = self.build_model(self.max_pad, self.train_Y.shape[2])
        (history, score) = self.learning(model, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_relu')
        plt = self.plt_init()
        self.draw_plot(history, score, '_relu')
        self.model.save('relu_model'+datetime.today().strftime("%Y%m%d"))
        print('relu model done')
        return self.model
    
    def build_learn_model_sig(self):
        print('building sigmoid model')
        split = int(len(self.train_X)*self.split_ratio)
        self.model_sig = self.build_model_sigmoid(self.max_pad, self.train_Y.shape[2])
        (history, score) = self.learning(model, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_relu')
        plt = self.plt_init()
        self.draw_plot(history, score, '_sigmoid')
        self.model.save('sigmoid_model'+datetime.today().strftime("%Y%m%d"))
        print('sigmoid model done')
        return self.model_sig
    
    def build_learn_model_bi(self):
        print('building bidirectional relu model')
        split = int(len(self.train_X)*self.split_ratio)
        self.model_bi = self.build_model_bi(self.max_pad, self.train_Y.shape[2])
        (history, score) = self.learning(model, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_relu')
        plt = self.plt_init()
        self.draw_plot(history, score, '_relu_bi')
        self.model.save('bi_relu_model')
        print('bidirectional relu model done'+datetime.today().strftime("%Y%m%d"))
        return self.model_bi
    
    def build_learn_model_sig_bi(self):
        print('building bidirectional sigmoid model')
        split = int(len(self.train_X)*self.split_ratio)
        self.model_sig_bi = self.build_model_sig_bi(self.max_pad, self.train_Y.shape[2])
        (history, score) = self.learning(model, self.train_X[:split],self.train_Y[:split],self.train_X[split:],self.train_Y[split:],'_relu')
        plt = self.plt_init()
        self.draw_plot(history, score, '_sigmoid_bi')
        self.model.save('bi_sigmoid_model'+datetime.today().strftime("%Y%m%d"))
        print('sigmoid model done')
        return self.model_sig_bi
    
    def predict_all(self):
        test = []
        ret = []
        pred = self.model.predict()
        ret.append(np.argmax(pred, axis=0))
        pred = self.model_sig.predict()
        ret.append(np.argmax(pred, axis=0))
        pred = self.model_bi.predict()
        ret.append(np.argmax(pred, axis=0))
        pred = self.model_sig_bi.predict()
        ret.append(np.argmax(pred, axis=0))
        return ret
    
    def precit_model(self):
        pred = self.model.predict()
        print(pred.shape)
        return np.argmax(pred, axis=0)
    
    def precit_model_sig(self):
        pred = self.model_sig.predict()
        print(pred.shape)
        return np.argmax(pred, axis=0)
    
    def precit_model_bi(self):
        pred = self.model_bi.predict()
        print(pred.shape)
        return np.argmax(pred, axis=0)
    
    def precit_model_sig_bi(self):
        pred = self.model_sig_bi.predict()
        print(pred.shape)
        return np.argmax(pred, axis=0)

In [104]:
"""MLP
    loadModel(self)
	data_manipulate(self)
	build_model_all(self)
	build_learn_model(self)
	build_learn_model_sig(self)
	build_learn_model_bi(self)
	build_learn_model_sig_bi(self)
	predict_all(self)
	predict_model(self)
	predict_model_sig(self)
	predict_model_bi(self)
	predict_model_sig_bi(self)
"""

'MLP\n    loadModel(self)\n\tdata_manipulate(self)\n\tbuild_model_all(self)\n\tbuild_learn_model(self)\n\tbuild_learn_model_sig(self)\n\tbuild_learn_model_bi(self)\n\tbuild_learn_model_sig_bi(self)\n\tpredict_all(self)\n\tpredict_model(self)\n\tpredict_model_sig(self)\n\tpredict_model_bi(self)\n\tpredict_model_sig_bi(self)\n'

In [106]:
#크롤링 각각 process 생성해서 돌린다

def crawling(obj, logger):
    print('Crawing starts...')
    ps = []
    crawling_list = [
        [joongang_crawling, 'joongang_crawling'],
        [joongang_cul,'joongang_cul'],
        [chosun_crawling, 'chosun_crawling'],
        [chosun_biz,'chosun_biz'],
        [donga_crawling,'donga_crawglin']
    ]
    
    if __name__ == '__main__':
        for func in crawling_list:
            p = Process(target = func[0])#, name=func[1])
            ps.append(p)
            p.start()

        for p in ps:
            p.join()

    cal_weight()
    obj.data_manipulate()
    return True

In [107]:
def init_program():
    multiprocessing.log_to_stderr()
    logger = multiprocessing.get_logger()
    logger.setLevel(logging.INFO)
    obj = MLP()
    init_mongo()
    return (obj, logger)

In [108]:
def base(obj, logger):
    #init_mysql()
    multiprocessing.log_to_stderr()
    logger = multiprocessing.get_logger()
    logger.setLevel(logging.INFO)
    build_ready = False
    recommand_ready = False
    p = None
    #  Menu
    while(1):
        menu = int(input("1 : crawling " +
                         "2 : load model" +
                         "3 : build model "+
                         "4 : start recommand program"))
        #  start crawling -- blocked
        if(menu == 1):
            train_ready = crawling(obj, logger)
            
        #    load model --blocked
        elif(menu == 2):
            if(obj.loadModel()==False):
                print("There's no saved model, please build model first")
                
        # build model -- blocked
        elif(menu == 3):
            if(train_ready == False):
                print("Training set not exist")
                continue
                
            p = int(input("1: build relu model\n" + 
                     "2 : build sigmoid model\n" +
                     "3 : build bidirectional relu model\n" +
                     "4 : build bidirectional sigmoid model\n" +
                     "5 : build all model\n"))
            if(p == 1):
                p = Process(target=obj.build_learn_model)
                p.start()
            elif(p == 2):
                p = Process(target=obj.build_learn_model_sig)
                p.start()
            elif(p == 3):
                p = Process(target=obj.build_learn_model_bi)
                p.start()
            elif(p == 4):
                p = Process(target=obj.build_learn_model_sig_bi)
                p.start()
            elif(p == 5):
                p = Process(target=obj.build_model_all)
                p.start()
        #  start recommand program
        elif(menu == 4):
            if(p is None):
                print('There is no model, please build model fisrt')
                continue
            subprocess.run(["python","./Recommand_v1.py"], stdout=subprocess.PIPE
                             , shell=True)
            print('recommanding process start')
            
    return

In [109]:
if __name__ == '__main__':
    (obj, logger) = init_program()
    base(obj, logger)    

1 : crawling 2 : load model3 : build model 4 : start recommand program1
Crawing starts...
calculating weight...


C:\Anaconda\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning:

"Twitter" has changed to "Okt" since KoNLPy v0.4.5.



ValueError: empty vocabulary; perhaps the documents only contain stop words